# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.

---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```

---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [ ]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

---
### 1.4. Instalar librerías necesarias

In [ ]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [ ]:
from __future__ import division
import os
import sys
import numpy as np
from PIL import Image
import gym
import tensorflow as tf
from tf_keras.models import Sequential
from tf_keras.layers import Dense, Flatten, Convolution2D, Permute
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import ModelIntervalCheckpoint, FileLogger
import tf_keras

#### Configuración base

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" 

os.environ["TF_USE_LEGACY_KERAS"] = "1"
sys.modules['tensorflow.keras'] = tf_keras



INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

In [ ]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE, Image.NEAREST).convert('L')
        return np.array(img).astype('uint8')

    def process_state_batch(self, batch):
        return batch.astype('float32') / 255.

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1. Implementación de la red neuronal
La red neuronal convolucional se compone de tres etapas, la transformación Permute permite que las capas convolucionales vean el movimiento, las capas convolucionales son filtros que buscan patrones, la etapa de decisión es la capa densa que consiste en 512 neuronas con activación ReLU

In [ ]:
def build_model(nb_actions):
    model = Sequential()
    model.add(Permute((2, 3, 1), input_shape=(WINDOW_LENGTH,) + INPUT_SHAPE))
    model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation='relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))
    return model


2. Implementación de la solución DQN

Se usa la versión Deterministic-v4 para que el agente aprenda de manera más eficiente, pero la fase de test en SpaceInvaders-v0
Se usa un decaimiento lineal de épsilon para pasar de exploración 100% a explotación 10%
Se realizan 1000000 pasos

Eliminamos casi todo el ruido de exploración para el test de entrenamiento con dqn.policy.eps = 0.005, esta fase de test se realiza sobre 100 partidas

In [ ]:

env_train = gym.make('SpaceInvadersDeterministic-v4')
nb_actions = env_train.action_space.n

model = build_model(nb_actions)
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()


train_policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., 
                                    value_min=.1, value_test=.01, nb_steps=1000000)

dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=train_policy, 
               memory=memory, processor=processor, enable_double_dqn=True,
               nb_steps_warmup=50000, target_model_update=10000)

dqn.compile(tf_keras.optimizers.legacy.Adam(learning_rate=.00025), metrics=['mae'])


callbacks = [ModelIntervalCheckpoint('dqn_checkpoint_{step}.h5f', interval=100000)]
callbacks += [FileLogger('dqn_training_log.json', interval=100)]

dqn.fit(env_train, callbacks=callbacks, nb_steps=1000000, visualize=False, verbose=2)
dqn.save_weights('dqn_checkpoint_final.h5f', overwrite=True)

In [ ]:

env_train.close()
env_test = gym.make('SpaceInvaders-v0', repeat_action_probability=0.0)


dqn.training = False
dqn.policy = EpsGreedyQPolicy() 
dqn.policy.eps = 0.005           

history = dqn.test(env_test, nb_episodes=100, visualize=False)

avg_history = np.mean(history.history['episode_reward'])
max_history = np.max(history.history['episode_reward'])
min_history = np.min(history.history['episode_reward'])

print(f"MEDIA: {avg_history:.2f} MEJOR PARTIDA: {max_history:.2f} PEOR PARTIDA:{min_history:.2f}")

2026-01-27 20:28:58.121190: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_10/kernel/Assign' id:2666 op device:{requested: '', assigned: ''} def:{{{node dense_10/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_10/kernel, dense_10/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2026-01-27 20:28:58.416246: W tensorflow/c/c_api.cc:305] Operation '{name:'total_21/Assign' id:2936 op device:{requested: '', assigned: ''} def:{{{node total_21/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](total_21, total_21/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in t

Testing for 100 episodes ...
Episode 1: reward: 15.000, steps: 644
Episode 2: reward: 24.000, steps: 960
Episode 3: reward: 13.000, steps: 567
Episode 4: reward: 30.000, steps: 1024
Episode 5: reward: 13.000, steps: 539
Episode 6: reward: 20.000, steps: 800
Episode 7: reward: 22.000, steps: 815
Episode 8: reward: 20.000, steps: 865
Episode 9: reward: 12.000, steps: 706
Episode 10: reward: 22.000, steps: 1070
Episode 11: reward: 22.000, steps: 899
Episode 12: reward: 21.000, steps: 864
Episode 13: reward: 11.000, steps: 687
Episode 14: reward: 26.000, steps: 909
Episode 15: reward: 29.000, steps: 1190
Episode 16: reward: 21.000, steps: 943
Episode 17: reward: 15.000, steps: 667
Episode 18: reward: 8.000, steps: 628
Episode 19: reward: 22.000, steps: 829
Episode 20: reward: 25.000, steps: 890
Episode 21: reward: 24.000, steps: 824
Episode 22: reward: 29.000, steps: 1142
Episode 23: reward: 26.000, steps: 861
Episode 24: reward: 22.000, steps: 935
Episode 25: reward: 25.000, steps: 1034
E

La red neuronal convolucional se compone de tres etapas, la transformación Permute permite que las capas convolucionales vean el movimiento, las capas convolucionales son filtros que buscan patrones, la etapa de decisión es la capa densa que consiste en 512 neuronas con activación ReLU

Se usa la versión Deterministic-v4 para que el agente aprenda de manera más eficiente, pero la fase de test en SpaceInvaders-v0
Se usa un decaimiento lineal de épsilon para pasar de exploración 100% a explotación 10%
Se realizan 1000000 pasos

Eliminamos casi todo el ruido de exploración para el test de entrenamiento con dqn.policy.eps = 0.005, esta fase de test se realiza sobre 100 partidas.

El rendimiento medio es 20.01, se observa que las partidas con mayor recompensa son las que tienen un número mayor de pasos como es normal. 

Parece que el agente a aprendido a atacar por llegar a 32 puntos, pero todavía tiene problemas con la densa
